# PyDentity Example - Data Owner Side

In this notebook we'll be initiating a connection to a remote Researcher through our Agents. This notebook has the following phases:

1. Pull in dependencies
2. Instatiate the controller for our Agent
3. Set up a data message listener running on a service on our controller
4. Use the controller to create an invitation from our agent
5. Copy the invitation output from 4 and move over to the Researcher notebook

<b>Carry on at the Researcher side</b>
    
12. Accept Request for Connection
13. Send Trust Ping to Researcher

<b>Carry on at the Researcher side</b>
    
<b>Begin PySyft Experiments</b>



### 1. Pull in dependencies

In [1]:
%autoawait
import time
import asyncio
import flask
from flask import request

IPython autoawait is `on`, and set to use `asyncio`


### 2. Instatiate the controller for our Agent

The arguments depend on how the aca-py agent was initiated. See the manage and docker-compose.yml files for more details.

In [2]:
from aries_basic_controller.aries_controller import AriesAgentController
    
WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 8022
WEBHOOK_BASE = ""
ADMIN_URL = "http://data-agent:8021"

# Based on the aca-py agent you wish to control
agent_controller = AriesAgentController(webhook_host=WEBHOOK_HOST, webhook_port=WEBHOOK_PORT,
                                       webhook_base=WEBHOOK_BASE, admin_url=ADMIN_URL, connections=True)

### 3. Set up a data message listener running as a service on our controller

In [3]:
def data_messages_handler(payload):
    connection_id = payload["connection_id"]
    asyncio.get_event_loop().create_task(agent_controller.connections.send_message(connection_id, "THIS IS MY RESPONSE - DATA AGENT"))
    print("Handle data messages ", payload, connection_id)


data_message_listener = {
    "handler": data_messages_handler,
    "topic": "basicmessages"
}

loop = asyncio.get_event_loop()
loop.create_task(agent_controller.listen_webhooks())

agent_controller.register_listeners([data_message_listener], defaults=True)

### 4. Use the controller to create an invitation from our agent


In [4]:
# Create Invitation
invite = await agent_controller.connections.create_invitation(alias="Will")
print(invite)

{'connection_id': 'b3431f7a-7a26-4425-88fa-742780a7b09d', 'invitation': {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation', '@id': '3cfc8191-010e-4982-8b17-5c9dd54e37d6', 'recipientKeys': ['Dy5uQGfxZQXG4QXKcMnJkW4birm5WqBJJ1Ls6mme7BkG'], 'label': 'Bob', 'serviceEndpoint': 'http://192.168.65.3:8020'}, 'invitation_url': 'http://192.168.65.3:8020?c_i=eyJAdHlwZSI6ICJkaWQ6c292OkJ6Q2JzTlloTXJqSGlxWkRUVUFTSGc7c3BlYy9jb25uZWN0aW9ucy8xLjAvaW52aXRhdGlvbiIsICJAaWQiOiAiM2NmYzgxOTEtMDEwZS00OTgyLThiMTctNWM5ZGQ1NGUzN2Q2IiwgInJlY2lwaWVudEtleXMiOiBbIkR5NXVRR2Z4WlFYRzRRWEtjTW5Ka1c0YmlybTVXcUJKSjFMczZtbWU3QmtHIl0sICJsYWJlbCI6ICJCb2IiLCAic2VydmljZUVuZHBvaW50IjogImh0dHA6Ly8xOTIuMTY4LjY1LjM6ODAyMCJ9'}


### 5. Copy the invitation output from 4 and move over to the Researcher notebook (localhost:8889) ^^^

### 12. Accept Request for Connection

In [5]:
# Print out accepted Invite and Researcher ID
print("Data Owner ID", invite["connection_id"])
data_connection_id = invite["connection_id"]

# Accept Request for Invite created
connection = await agent_controller.connections.accept_request(data_connection_id)
print("ACCEPT REQUEST")
print(connection)

Data Owner ID b3431f7a-7a26-4425-88fa-742780a7b09d
True
ACCEPT REQUEST
{'created_at': '2020-06-26 11:38:55.560696Z', 'invitation_key': 'Dy5uQGfxZQXG4QXKcMnJkW4birm5WqBJJ1Ls6mme7BkG', 'routing_state': 'none', 'initiator': 'self', 'invitation_mode': 'once', 'updated_at': '2020-06-26 11:39:45.256521Z', 'their_label': 'Alice', 'connection_id': 'b3431f7a-7a26-4425-88fa-742780a7b09d', 'my_did': 'Mfc76nvc5xzWgGWoBjJcuL', 'state': 'response', 'accept': 'manual', 'their_did': 'NAN12ecuQ2Kni6FoD9j3Gd'}


### 13. Send Trust Ping, return Ping on Researcher side to activate the conneciton

In [6]:
trust_ping = await agent_controller.connections.trust_ping(data_connection_id, "hello")
print(trust_ping)

{'thread_id': 'b591d41a-52fe-4b84-8d11-871cec1f3edf'}


####  13.1 Check if connection is active

In [7]:
connection = await agent_controller.connections.get_connection(data_connection_id)
print(connection)

{'created_at': '2020-06-26 11:38:55.560696Z', 'invitation_key': 'Dy5uQGfxZQXG4QXKcMnJkW4birm5WqBJJ1Ls6mme7BkG', 'routing_state': 'none', 'initiator': 'self', 'invitation_mode': 'once', 'updated_at': '2020-06-26 11:39:48.139418Z', 'their_label': 'Alice', 'connection_id': 'b3431f7a-7a26-4425-88fa-742780a7b09d', 'my_did': 'Mfc76nvc5xzWgGWoBjJcuL', 'state': 'active', 'accept': 'manual', 'their_did': 'NAN12ecuQ2Kni6FoD9j3Gd'}


####  13.2 Send a basic message over e2e channel

In [8]:
message = await agent_controller.connections.send_message(data_connection_id,"hello from data owner world! WOOPWOOP")
print("BASIC MESSAGE - DATA -> RESEARCH")
print(message)

BASIC MESSAGE - DATA -> RESEARCH
{}


## Begin Syft Worker Experimentation

We'd like to extend the Baseworker class to use the basic messaging service in the Aries controller. We are beginning this below. The plan is to turn the binary into a string so that it can be communicated over the Aries Basic Message protocol. We're trying to send a syft tensor, x, over the channel. If the other side receives it, itll be able to interpret it with it's own Baseworker. 

In [9]:
import torch
import syft as sy
from syft.serde import deserialize
from syft.serde import serialize
from syft.workers.base import BaseWorker


class AriesWorker(BaseWorker):
    def _send_msg(self, message: bin, location: BaseWorker) -> bin:
      return self._recv_msg(message)

    def _recv_msg(self, message: bin) -> bin:
      result = deserialize(self.recv_msg(message))
      print(result)
      message = asyncio.get_event_loop().create_task(agent_controller.connections.send_message(data_connection_id, result ))
      print(message)
      return result

Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/opt/conda/lib/python3.7/site-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.15.3.so'


In [10]:
hook = sy.TorchHook(torch)
bob = AriesWorker(hook, id="bob")

In [11]:
x = torch.tensor([1,2,3,4,5])




In [12]:


x_ptr = x.send(bob)



None
<Task pending coro=<ConnectionsController.send_message() running at /opt/conda/lib/python3.7/site-packages/aries_basic_controller/connections_controller.py:94>>


TypeError: 'NoneType' object is not subscriptable